In [ ]:
# Data Wrangling
import pandas as pd

from pandas import Series, DataFrame
import numpy as np

# Visualization
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
%matplotlib inline
rc('font', family='malgun gothic')
import seaborn as sns
%matplotlib inline

# EDA

# Preprocessing & Feature Engineering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import StratifiedKFold
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold

from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from random import uniform
from scipy.stats import gmean
from scipy.stats import randint

# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Modeling
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.base import ClassifierMixin
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostClassifier
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import ExtraTreesClassifier
# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

# Utility
import os
import time
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean
from sklearn.decomposition import PCA
import datetime

In [ ]:
# Read Data
X = pd.read_csv('X.csv',)
y = pd.read_csv('y.csv')
test_na = pd.read_csv('test.csv')

In [ ]:
l = []
for col in X.columns:
    if X[col].nunique() != 2:
        l.append(col)

In [ ]:
# 데이터 타입 변환
X = X.astype('str')
test_na = test_na.astype('str')

X[['전체회차','현재회차','서비스일자','접수일','최초서비스일']] = X[['전체회차','현재회차','서비스일자','접수일','최초서비스일']].astype('int')
test_na[['전체회차','현재회차','서비스일자','접수일','최초서비스일']] = test_na[['전체회차','현재회차','서비스일자','접수일','최초서비스일']].astype('int')

## Modeling

In [ ]:
X.columns

In [ ]:
# 범주형 변수
cat_list = ['장기서비스여부', '기존고객여부', '서비스주소',  
       'betweentime', 'ser_add_dae', 'ser_add_so', 
       '종료여부', '고객가입일', '고객서비스월', 
       '우선청소', '부재중여부', '매니저이동방법', '근무가능지역', '부재중서비스가능여부', '매니저생년월일',
       'CS교육이수여부', '청소교육이수여부', 'so_가능', 'dae_가능', '지역_가능', '나이대',
       '전체회차','현재회차',"잔여회차", '서비스일자', '접수일', '최초서비스일', "cus","man",'cus_man']

In [ ]:
# Validation
kfold = KFold(n_splits=5, shuffle = True, random_state=0)
answer = []

catf = CatBoostClassifier(cat_features=cat_list, random_state = 20182830)
scores = cross_val_score(catf , X[cat_list], y, cv=kfold, scoring='roc_auc')
answer.append(scores)
print(answer)

In [ ]:
# Fitting CatBoostClassifier
catf = CatBoostClassifier(cat_features=cat_list, random_state = 20182830)
catf.fit(X[cat_list], y)

In [ ]:
X.dtypes

In [ ]:
def plot_feature_importance(importance, names, model_type):
    
    feature_importance = np.array(importance)
    feature_names = np.array(names)
    
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    
    plt.figure(figsize=(10,8))
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    plt.title(model_type + 'FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')

In [ ]:
# Feature Importance
plot_feature_importance(catf.get_feature_importance(), X[cat_list].columns,'CatBOOST')

In [ ]:
# Submission
sub = pd.read_csv("data/sample_submission_final.csv")
pred_tr = catf.predict_proba(test_na[cat_list])
sub["pred"] = pd.DataFrame(pred_tr)[1]

sub.to_csv("sota_test_2.csv",index = False)